
# 中華隊成績查詢與分析工具
**Credit by Simon Liu**

此 Notebook 使用生成式 AI 和工具整合來查詢棒球選手的比賽成績並進行數據分析。


In [1]:
## 安裝必要的套件
# 安裝 LangChain 社群工具包和 Google Generative AI 模組。
# 安裝必要的套件
!pip install -q langchain_community langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.7 MB/s eta 0:00:00


# Dataset

In [2]:
# Download Dataset from huggingface
## Please see the dataset here: https://huggingface.co/datasets/Simon-Liu/premier-12-chinese-taipei-performance-data/

!wget https://huggingface.co/datasets/Simon-Liu/premier-12-chinese-taipei-performance-data/resolve/main/%E4%B8%AD%E8%8F%AF%E9%9A%8A_bat_data_with_chinese_names.csv
!wget https://huggingface.co/datasets/Simon-Liu/premier-12-chinese-taipei-performance-data/resolve/main/%E4%B8%AD%E8%8F%AF%E9%9A%8A_field_data_with_chinese_names.csv
!wget https://huggingface.co/datasets/Simon-Liu/premier-12-chinese-taipei-performance-data/resolve/main/%E4%B8%AD%E8%8F%AF%E9%9A%8A_pitch_data_with_chinese_names.csv

--2024-12-26 02:59:24--  https://huggingface.co/datasets/Simon-Liu/premier-12-chinese-taipei-performance-data/resolve/main/%E4%B8%AD%E8%8F%AF%E9%9A%8A_bat_data_with_chinese_names.csv
Resolving huggingface.co (huggingface.co)... 3.165.102.22, 3.165.102.58, 3.165.102.128, ...
Connecting to huggingface.co (huggingface.co)|3.165.102.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1325 (1.3K) [text/plain]
Saving to: ‘中華隊_bat_data_with_chinese_names.csv’

中華隊_bat_data_wit 100%[===================>]   1.29K  --.-KB/s    in 0s      

2024-12-26 02:59:24 (410 MB/s) - ‘中華隊_bat_data_with_chinese_names.csv’ saved [1325/1325]

--2024-12-26 02:59:24--  https://huggingface.co/datasets/Simon-Liu/premier-12-chinese-taipei-performance-data/resolve/main/%E4%B8%AD%E8%8F%AF%E9%9A%8A_field_data_with_chinese_names.csv
Resolving huggingface.co (huggingface.co)... 3.165.102.22, 3.165.102.58, 3.165.102.128, ...
Connecting to huggingface.co (huggingface.co)|3.165.102.22|:443... conne

# Model

In [3]:
## 設定 Google API
# 設定 Google API 金鑰，準備初始化 Google 生成式 AI 模型。
import os
from google.colab import userdata

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [4]:
## 初始化生成式 AI 模型
# 使用 Google Generative AI 的模型來處理查詢。
from langchain_google_genai import ChatGoogleGenerativeAI

# 初始化語言模型
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-8b",
    temperature=0,
)

# LLM with tool - Function Calling

In [5]:
## 定義打擊成績查詢工具
# 使用 LangChain 的工具功能定義搜尋特定選手打擊數據的工具。
import pandas as pd
import json
from langchain_core.tools import tool

@tool
def search_bat_player_stats(player_name: str) -> str:
    """
    Search for player statistics by name in the CSV file.
    Returns raw data in JSON format.
    """
    try:
        # 讀取 CSV 文件
        file_path = "/content/中華隊_bat_data_with_chinese_names.csv"
        df = pd.read_csv(file_path)

        # 搜尋球員
        player_data = df[df['player_chinese'] == player_name]

        if player_data.empty:
            return json.dumps({"error": "Player not found."})

        # 將結果轉換為 JSON 格式
        result = player_data.to_dict(orient="records")
        return json.dumps(result, ensure_ascii=False)

    except Exception as e:
        return json.dumps({"error": str(e)})

In [6]:
## 測試打擊成績查詢工具
# 測試 `search_bat_player_stats` 工具，查詢張政禹的打擊數據。
%%time

search_bat_player_stats.invoke({"player_name": "張政禹"})

CPU times: user 12.2 ms, sys: 1.92 ms, total: 14.1 ms
Wall time: 36.3 ms


'[{"球員": "CHANG\\nCheng-Yu", "AB": 7, "R": 1, "H": 1, "2B": 0, "3B": 0, "HR": 0, "RBI": 1, "TB": 1, "AVG": 0.143, "SLG": 0.143, "OBP": 0.143, "OPS": 0.286, "BB": 0, "HBP": 0, "SO": 1, "GDP": 0, "SF": 0, "SH": 0, "SB": 1, "CS": 0, "player_chinese": "張政禹"}]'

In [7]:
## 綁定工具與語言模型
# 將 `search_bat_player_stats` 工具綁定到語言模型，方便整合操作。
llm_with_tools = llm.bind_tools([search_bat_player_stats])

In [8]:
## 測試語言模型與工具的整合
# 測試整合後的工具，詢問張政禹的比賽成績並檢視結果。
%%time

msg = llm_with_tools.invoke("張政禹選手的成績？")
msg.tool_calls

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 API Key not found. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API Key not found. Please pass a valid API key."
]

# AI Agent with three tools

In [9]:
## 定義自訂 Prompt 模板
# 定義用於生成式 AI 和工具的 Prompt，包含佔位符以便替換。
from langchain_core.prompts import ChatPromptTemplate

# 定義包含佔位符的字串模板
template = """
================================ System Message ================================

這是一個可以查詢 2024 12強棒球賽，中華隊投球、打擊、守備數據庫，
你是一個專業的數據查詢和分析助手。你可以使用工具來查詢數據並幫助用戶完成額外計算。

工具分成：
1. 打擊數據成績
2. 投球數據成績
3. 守備數據成績

當用戶詢問問題時：
- 首先使用工具查詢棒球員的相關數據。
- 然後完成所需的計算。
- 最後以自然語言回答用戶的問題。

現在準備好處理用戶的請求。

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}
"""

# 使用 from_template 方法將字串轉換為 ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(template)
prompt.pretty_print()

================================ Human Message =================================


================================ System Message ================================

這是一個可以查詢 2024 12強棒球賽，中華隊投球、打擊、守備數據庫，
你是一個專業的數據查詢和分析助手。你可以使用工具來查詢數據並幫助用戶完成額外計算。

工具分成：
1. 打擊數據成績
2. 投球數據成績
3. 守備數據成績

當用戶詢問問題時：
- 首先使用工具查詢棒球員的相關數據。
- 然後完成所需的計算。
- 最後以自然語言回答用戶的問題。

現在準備好處理用戶的請求。

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}



In [10]:
## 定義守備成績查詢工具
# 定義搜尋守備成績數據的工具，擴展數據查詢能力。
import pandas as pd
import json
from langchain_core.tools import tool

@tool
def search_field_player_stats(player_name: str) -> str:
    """
    Search for player statistics by name in the CSV file.
    Returns raw data in JSON format.
    """
    try:
        # 讀取 CSV 文件
        file_path = "/content/中華隊_field_data_with_chinese_names.csv"
        df = pd.read_csv(file_path)

        # 搜尋球員
        player_data = df[df['player_chinese'] == player_name]

        if player_data.empty:
            return json.dumps({"error": "Player not found."})

        # 將結果轉換為 JSON 格式
        result = player_data.to_dict(orient="records")
        return json.dumps(result, ensure_ascii=False)

    except Exception as e:
        return json.dumps({"error": str(e)})

In [11]:
## 定義投球成績查詢工具
# 定義搜尋投球成績數據的工具，提供完整的投手數據支持。
import pandas as pd
import json
from langchain_core.tools import tool

@tool
def search_pitch_player_stats(player_name: str) -> str:
    """
    Search for player statistics by name in the CSV file.
    Returns raw data in JSON format.
    """
    try:
        # 讀取 CSV 文件
        file_path = "/content/中華隊_pitch_data_with_chinese_names.csv"
        df = pd.read_csv(file_path)

        # 搜尋球員
        player_data = df[df['player_chinese'] == player_name]

        if player_data.empty:
            return json.dumps({"error": "Player not found."})

        # 將結果轉換為 JSON 格式
        result = player_data.to_dict(orient="records")
        return json.dumps(result, ensure_ascii=False)

    except Exception as e:
        return json.dumps({"error": str(e)})

In [12]:
## 載入代理工具執行模組
# 載入 LangChain 的代理執行功能模組。
from langchain.agents import AgentExecutor, create_tool_calling_agent

## 整合所有工具
# 將打擊、守備和投球數據查詢工具整合到工具清單中。
tools = [search_bat_player_stats, search_field_player_stats, search_pitch_player_stats]

## 創建工具代理
# 建立代理工具系統，將語言模型和工具清單整合以提供查詢能力。
# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

## 建立代理執行器
# 設定代理執行器，允許使用工具代理執行複雜查詢。
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
    max_iterations=5  # Example limit
)

# User can ask the question here

In [13]:
## 測試代理執行器
# 使用代理執行器查詢王志煊的投球成績，並計算滾飛比。
%%time

result = agent_executor.invoke(
    {
        "input": "請問王志煊的投球成績，並幫我計算滾飛比？"
    }
)



> Entering new AgentExecutor chain...


ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 API Key not found. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API Key not found. Please pass a valid API key."
]

In [14]:
## 列印結果
# 輸出查詢結果到控制台。
print(result['output'])

NameError: name 'result' is not defined

In [ ]:
## 測試其他查詢
# 測試代理執行器查詢林安可的打擊狀況，檢視結果。
%%time

result = agent_executor.invoke(
    {
        "input": "請問林安可這次賽會的打擊狀況如何？"
    }
)

In [ ]:
## 格式化並列印輸出
# 將輸出格式化後列印，提高結果的可讀性。
# 印出結果
print(result['output'].replace('。', '。\n'))

# Use Google Mesop python package application UI.

In [ ]:
!pip install mesop

In [ ]:
import mesop as me
import mesop.labs as mel

me.colab_run()

In [ ]:
@me.page(path="/chat")
def chat():
  mel.chat(transform)

def transform(prompt: str, history: list[mel.ChatMessage]) -> str:
    result = agent_executor.invoke(
        {
            "input": prompt
        }
    )

    return result['output']

In [ ]:
me.colab_show(path="/chat", height = '400')